# Zero-Shot Classification Playground (Local Models)

Use this notebook to experiment with Hugging Face `zero-shot-classification` models while storing model files **inside this project workspace**.

This is helpful when you want to:
- keep tight control over model assets,
- work with predictable local paths,
- compare models for real developer workflows (issue triage, commit categorization, support routing).

In [1]:
from dotenv import load_dotenv
load_dotenv(dotenv_path=".env.local")

import os
os.environ["HF_HUB_CACHE"] = "./models/huggingface/_cache"
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"


In [2]:
from pathlib import Path
from typing import List, TypedDict

# If needed in a fresh environment, uncomment and run:
# %pip install -q transformers torch

from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import torch

In [3]:
def get_local_zero_shot_classifier(
        model_id: str,
        models_folder: str = "./models/huggingface",
        force_redownload: bool = False,
):
    """
    Download a zero-shot model into this workspace and return an inference pipeline.

    Model files are stored at:
        {workspace_root}/{models_folder}/{model_id}
    """
    local_model_dir = Path(models_folder) / model_id.replace("/", "--")
    local_model_dir.mkdir(parents=True, exist_ok=True)

    has_model_files = (local_model_dir / "config.json").exists()

    if force_redownload or not has_model_files:
        tokenizer = AutoTokenizer.from_pretrained(model_id)
        model = AutoModelForSequenceClassification.from_pretrained(model_id)

        tokenizer.save_pretrained(local_model_dir)
        model.save_pretrained(local_model_dir)

    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Loading model \"{model_id}\" from \"{local_model_dir}\" on {device}")

    classifier = pipeline(
        task="zero-shot-classification",
        model=str(local_model_dir),
        tokenizer=str(local_model_dir),
        device=device
    )

    return classifier

In [4]:
# Developer-friendly example scenarios
class Experiment(TypedDict):
    title: str
    text: str
    labels: List[str]
    multi_label: bool


def run_zero_shot_experiment(model_id: str, experiments: List[Experiment]) -> None:
    classifier = get_local_zero_shot_classifier(model_id)

    for exp in experiments:
        result = classifier(
            sequences=exp["text"],
            candidate_labels=exp["labels"],
            multi_label=exp["multi_label"],
        )

        ranked_predictions = [
            {"label": label, "score": float(score)}
            for label, score in zip(result["labels"], result["scores"])
        ]

        print()
        print(f"{'=' * 10} [{model_id}] {'=' * 10}")
        print(exp["title"])
        print(f"Text: {exp["text"]}")

        for p in ranked_predictions:
            print(f"  - {p['label']:<20} score={p['score']:.4f}")

In [5]:
supported_models = {
    "facebook": {"bart_large_mnli": "facebook/bart-large-mnli"},
    "cross_encoder": {"nli_deberta_v3_base": "cross-encoder/nli-deberta-v3-base"},
}

examples: List[Experiment] = [
    {
        "title": "1) Stock market news sentiment",
        "text": "The tech giant reported quarterly revenue below analyst expectations, causing a dip in shares.",
        "labels": ["positive", "negative", "neutral"],
        "multi_label": False,
    },
    {
        "title": "2) Political speech topic detection",
        "text": "We must strengthen international alliances to secure trade routes and global stability.",
        "labels": ["economy", "environment", "foreign policy", "education", "healthcare"],
        "multi_label": True,
    },
    {
        "title": "3) Restaurant review categorization",
        "text": "The decor was cozy, the service was quick, but the main course lacked seasoning.",
        "labels": ["food quality", "service", "ambiance", "price"],
        "multi_label": True,
    },
    {
        "title": "4) Music genre classification",
        "text": "Heavy guitar riffs, pounding drums, and aggressive vocals drive this intense track.",
        "labels": ["rock", "pop", "jazz", "classical", "hip hop", "electronic"],
        "multi_label": False,
    },
    {
        "title": "5) Financial report topic detection",
        "text": "The central bank decided to adjust interest rates, and job creation slowed compared to last quarter, affecting market confidence.",
        "labels": ["monetary policy", "employment", "inflation", "trade", "stock market"],
        "multi_label": True,
    },
    {
        "title": "6) Political stance classification",
        "text": "The new legislation limits government oversight of social media platforms.",
        "labels": ["support", "oppose", "neutral"],
        "multi_label": False,
    },
    {
        "title": "7) Menu item type detection",
        "text": "A small portion of grilled octopus with a lemon vinaigrette.",
        "labels": ["appetizer", "main course", "dessert", "beverage", "snack"],
        "multi_label": False,
    },
    {
        "title": "8) Music mood classification",
        "text": "Soft piano and ambient synths create a calm, introspective atmosphere.",
        "labels": ["happy", "sad", "angry", "relaxed", "romantic"],
        "multi_label": False,
    },
    {
        "title": "9) Investment advisory query",
        "text": "I want to know the safest way to diversify my retirement portfolio this year.",
        "labels": ["tax", "investment strategy", "retirement planning", "loans", "insurance"],
        "multi_label": True,
    },
    {
        "title": "10) Politician statement classification",
        "text": "Expanding public transportation will help reduce traffic congestion and pollution in cities.",
        "labels": ["environment", "infrastructure", "healthcare", "economy", "education"],
        "multi_label": True,
    },
    {
        "title": "11) Food blog topic detection",
        "text": "Tips for fermenting vegetables at home to create healthy, flavorful dishes.",
        "labels": ["baking", "cooking techniques", "healthy recipes", "restaurant reviews", "desserts"],
        "multi_label": True,
    },
]

In [6]:
run_zero_shot_experiment(supported_models["facebook"]["bart_large_mnli"], examples)

Loading model "facebook/bart-large-mnli" from "models\huggingface\facebook--bart-large-mnli" on cuda


Loading weights:   0%|          | 0/515 [00:00<?, ?it/s]


========== [facebook/bart-large-mnli] ==========
1) Stock market news sentiment
Text: The tech giant reported quarterly revenue below analyst expectations, causing a dip in shares.
  - negative             score=0.9754
  - neutral              score=0.0196
  - positive             score=0.0050

========== [facebook/bart-large-mnli] ==========
2) Political speech topic detection
Text: We must strengthen international alliances to secure trade routes and global stability.
  - foreign policy       score=0.9758
  - economy              score=0.5974
  - environment          score=0.0125
  - healthcare           score=0.0008
  - education            score=0.0004

========== [facebook/bart-large-mnli] ==========
3) Restaurant review categorization
Text: The decor was cozy, the service was quick, but the main course lacked seasoning.
  - ambiance             score=0.9776
  - service              score=0.8033
  - food quality         score=0.5794
  - price                score=0.0417

========

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset



========== [facebook/bart-large-mnli] ==========
9) Investment advisory query
Text: I want to know the safest way to diversify my retirement portfolio this year.
  - retirement planning  score=0.9908
  - investment strategy  score=0.9527
  - loans                score=0.0783
  - insurance            score=0.0478
  - tax                  score=0.0190

========== [facebook/bart-large-mnli] ==========
10) Politician statement classification
Text: Expanding public transportation will help reduce traffic congestion and pollution in cities.
  - environment          score=0.8607
  - infrastructure       score=0.2554
  - economy              score=0.2349
  - healthcare           score=0.0001
  - education            score=0.0001

========== [facebook/bart-large-mnli] ==========
11) Food blog topic detection
Text: Tips for fermenting vegetables at home to create healthy, flavorful dishes.
  - healthy recipes      score=0.9618
  - cooking techniques   score=0.8900
  - desserts             score

In [7]:
run_zero_shot_experiment(supported_models["cross_encoder"]["nli_deberta_v3_base"], examples)

Loading model "cross-encoder/nli-deberta-v3-base" from "models\huggingface\cross-encoder--nli-deberta-v3-base" on cuda


Loading weights:   0%|          | 0/202 [00:00<?, ?it/s]


========== [cross-encoder/nli-deberta-v3-base] ==========
1) Stock market news sentiment
Text: The tech giant reported quarterly revenue below analyst expectations, causing a dip in shares.
  - negative             score=0.9957
  - neutral              score=0.0037
  - positive             score=0.0007

========== [cross-encoder/nli-deberta-v3-base] ==========
2) Political speech topic detection
Text: We must strengthen international alliances to secure trade routes and global stability.
  - foreign policy       score=0.9999
  - economy              score=0.8921
  - environment          score=0.0548
  - education            score=0.0001
  - healthcare           score=0.0001

========== [cross-encoder/nli-deberta-v3-base] ==========
3) Restaurant review categorization
Text: The decor was cozy, the service was quick, but the main course lacked seasoning.
  - ambiance             score=0.9998
  - service              score=0.9990
  - food quality         score=0.0210
  - price           